<a href="https://colab.research.google.com/github/ankitarm/Data_Engineer_Scenario/blob/main/Data_Engineer_Mocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mock 1 : https://www.youtube.com/watch?v=-iMhSpP77F8

##Q2. Please explain what Kinesis is and its role.
### **What is AWS Kinesis?**  
AWS Kinesis is a **fully managed, scalable, real-time data streaming service** provided by Amazon Web Services (AWS). It is designed to **collect, process, and analyze streaming data** (e.g., logs, metrics, transactions, social media feeds) in real time.  

Kinesis is often compared to **Apache Kafka** (an open-source distributed event streaming platform), but unlike Kafka, Kinesis is **fully managed by AWS**, eliminating the need to handle cluster setup, scaling, or maintenance.

---

 **Key Components of Kinesis**  
Kinesis consists of multiple services, each serving different streaming needs:  

1. **Kinesis Data Streams**  
   - **Role:** Ingests and stores real-time data records (e.g., clickstreams, IoT sensor data).  

2. **Kinesis Data Firehose**  
   - **Role:** **Automatically loads streaming data into AWS destinations** (S3, Redshift, Elasticsearch, etc.).  


##Q3. Can you elaborate more on diff between kafka and kinesis


| **Feature**               | **Apache Kafka**                                   | **AWS Kinesis**                                  |
|---------------------------|---------------------------------------------------|-------------------------------------------------|
| **Managed Service**       | Self-hosted (requires setup/maintenance)          | Fully managed by AWS                            |
| **Scalability**           | Manual scaling (add brokers/partitions)           | Auto-scaling (adjust shards dynamically)        |
| **Latency**               | Ultra-low (~10ms)                                 | Higher (~200ms)                                 |
| **Durability**            | Configurable replication (user-managed)           | Built-in replication across AZs                 |
| **Pricing Model**         | Open-source (infrastructure costs apply)          | Pay per shard/hour + data volume               |
| **Data Retention**         | Configurable (days to years)                      | Fixed (1–365 days)                              |
| **Throughput**            | Higher (supports 100k+ msgs/sec per partition)    | Lower (1MB/sec or 1k msgs/sec per shard)       |
| **Partitions/Shards**     | Partitions (user-managed)                         | Shards (AWS-managed)                            |
| **Integrations**          | Multi-cloud, on-prem, hybrid                      | AWS-native (Lambda, S3, Redshift, etc.)        |
| **Use Cases**             | High-throughput, low-latency, cross-platform apps | Real-time analytics, AWS-centric pipelines     |
| **Setup Complexity**      | High (requires tuning, monitoring)                | Low (serverless, minimal configuration)        |
| **Security**              | Custom (SSL, SASL, Kerberos)                      | AWS IAM, KMS encryption                        |
| **Consumer Model**        | Pull-based (consumers request data)               | Push/pull hybrid (Kinesis Client Library)      |

---

### **Key Takeaways:**
1. **Choose Kafka** if you need:  
   - Cross-platform deployment (e.g., hybrid cloud).  
   - Ultra-low latency (e.g., financial trading).  
   - Full control over infrastructure.  

2. **Choose Kinesis** if you:  
   - Prefer a serverless, AWS-native solution.  
   - Need quick setup without managing clusters.  
   - Use AWS services (e.g., Lambda, Firehose).  

Which to Choose?
Need long retention (years) or flexibility? → Kafka.

Want simplicity and AWS integration? → Kinesis (but max 1 year).

##Q4. **Kafka vs. Kinesis: Producer-Consumer or Pub/Sub?**  


| **Model Aspect**       | **Apache Kafka**                                   | **AWS Kinesis**                                  |
|------------------------|---------------------------------------------------|-------------------------------------------------|
| **Primary Model**      | **Pub/Sub** (with consumer groups)                | **Producer-Consumer** (with shard-level parallelism) |
| **Data Flow**          | - Producers → **Topics** → Consumers (pull-based) | - Producers → **Streams/Shards** → Consumers (push/pull hybrid) |
| **Subscription Style** | - Consumers **subscribe to topics** (flexible)    | - Consumers **read from shards** (fixed mapping) |
| **Message Delivery**   | - **Pull-based** (consumers request data)         | - **Push (KCL) or Pull (Lambda, SDKs)**         |
| **Parallelism**        | - **Partitions** allow parallel consumers         | - **Shards** enable parallel processing         |
| **Decoupling**         | - High (multiple consumer groups per topic)       | - Medium (consumers compete for shard leases)   |

---

### **Key Differences**  
1. **Kafka = Pub/Sub with Flexibility**  
   - Producers publish to **topics**, and **consumer groups** subscribe.  
   - Multiple independent consumer groups can read the same data (e.g., one for analytics, another for alerts).  
   - Example:  
     ```python
     # Kafka Pub/Sub Example
     producer.send("orders-topic", order_data)  # Publisher
     consumer.subscribe(["orders-topic"])      # Subscriber
     ```

2. **Kinesis = Producer-Consumer with Scalability**  
   - Producers write to **shards**, and consumers **lease shards** for processing.  
   - Each shard supports **one consumer at a time** (no native fan-out).  
   - Example:  
     ```python
     # Kinesis Producer-Consumer Example
     kinesis.put_record(StreamName="logs-stream", Data=log_data)  # Producer
     shard_reader = kinesis.get_shard_iterator(...)               # Consumer
     ```



##Q5. What is the difference between producer-consumer and publisher-subscriber?

### **Producer-Consumer vs. Publisher-Subscriber**  

These are two fundamental messaging patterns in distributed systems, differing in **coupling**, **scalability**, and **message delivery** mechanics.

---

## **1. Producer-Consumer Model**  
**Definition:**  
- A **synchronous, point-to-point** messaging pattern.  
- **Producers** send messages to a **queue**, and **consumers** pull messages from it.  
- Each message is processed by **exactly one consumer**.  

**Key Traits:**  
✅ **Point-to-Point (1:1)** – Only one consumer gets each message.  
✅ **Tight Coupling** – Producers/consumers must know the queue.  
✅ **Pull-Based** – Consumers request messages when ready.  
✅ **Order Guaranteed** – FIFO (First-In-First-Out) by default.  

**Example:**  
- A **task queue** (e.g., RabbitMQ, SQS).  
- Workers processing orders from an e-commerce system.  

**Diagram:**  
```
Producer → [ Queue ] → Consumer
```

---

## **2. Publisher-Subscriber (Pub/Sub) Model**  
**Definition:**  
- An **asynchronous, broadcast** messaging pattern.  
- **Publishers** send messages to a **topic**, and **subscribers** receive them.  
- Each message is processed by **all interested subscribers**.  

**Key Traits:**  
✅ **Broadcast (1:N)** – Multiple subscribers receive the same message.  
✅ **Loose Coupling** – Publishers/subscribers only know the topic, not each other.  
✅ **Push-Based** – Messages are pushed to subscribers (or pulled via polling).  
✅ **Dynamic Scaling** – Subscribers can join/leave anytime.  

**Example:**  
- **Stock market feeds** (multiple apps listen to price updates).  
- **Event-driven microservices** (e.g., Kafka, AWS SNS).  

**Diagram:**  
```
Publisher → [ Topic ] → Subscriber 1  
                     → Subscriber 2  
                     → Subscriber 3
```

---

## **Key Differences Summary**  
| **Aspect**          | **Producer-Consumer**                | **Publisher-Subscriber**             |
|----------------------|--------------------------------------|--------------------------------------|
| **Messaging Style**  | Point-to-point (1:1)                 | Broadcast (1:N)                      |
| **Coupling**         | Tight (queue-aware)                  | Loose (topic-based)                  |
| **Delivery**         | Pull-based (consumer-driven)         | Push-based (or polled)               |
| **Scalability**      | Limited (competing consumers)        | High (independent subscribers)       |
| **Use Cases**        | Task queues, ordered processing      | Real-time notifications, event buses |

---

### **Which to Choose?**  
- **Use Producer-Consumer** when:  
  - You need **exactly-once processing** (e.g., order fulfillment).  
  - Order matters (e.g., FIFO queues).  

- **Use Pub/Sub** when:  
  - Multiple systems need **the same data** (e.g., analytics + alerts).  
  - You want **decoupled, event-driven architectures**.  

**Hybrid Systems (Kafka/Kinesis):**  
- Both models can coexist (e.g., Kafka uses **consumer groups** for Pub/Sub-like behavior while maintaining ordering per partition).  

Would you like a real-world analogy (e.g., postal service vs. radio broadcast)?

##Q6. Why are Kafka or Kinesis used instead of simple messaging queues?

### **Message Ordering in Kafka vs. Kinesis**  

#### **✅ Kafka**  
- **Ordered per partition**: Messages in the same partition are strictly ordered (FIFO).  
- **No global order**: Across partitions, order is **not** guaranteed unless using a single partition (bottleneck).  
- **Key-based routing**: Same key → same partition → preserves order for related messages.  
- **Use case**: E.g., ensuring all events for a user ID are processed in sequence.  

#### **✅ Kinesis**  
- **Ordered per shard**: Messages in the same shard are strictly ordered.  
- **No cross-shard order**: Like Kafka, order is lost across shards.  
- **Partition key routing**: Same key → same shard → ordered sequence.  
- **Use case**: E.g., processing financial transactions in exact arrival order per account.  

#### **⚠️ Key Limitation**  
- Both **cannot guarantee global order** (across partitions/shard) without sacrificing scalability.  


##Q7. Can messages be ordered in Kafka or Kinesis?

### **Message Ordering in Kafka vs. Kinesis**  

**Short Answer:**  
Yes, but **only within a partition (Kafka) or shard (Kinesis)**. Global ordering across all partitions/shard is **not guaranteed**.  

#### **Kafka**  
- **Order Guarantee**: Strictly ordered **per partition**.  
  - Example: If messages M1, M2, M3 are sent to **Partition 1**, they’ll be consumed in order.  
  - Cross-partition order is **not guaranteed** (e.g., M1 in Partition 1 and M4 in Partition 2 may arrive out of order).  
- **How to Preserve Order**:  
  - Use a **message key** (e.g., `user_id`) to ensure related messages go to the same partition.  

#### **Kinesis**  
- **Order Guarantee**: Strictly ordered **per shard**.  
  - Example: Messages in **Shard A** are ordered, but Shard A vs. Shard B may deliver data out of sequence.  
- **How to Preserve Order**:  
  - Use a **partition key** (e.g., `device_id`) to route related messages to the same shard.  

---

### **When Order Matters**  
- **Use Cases**:  
  - **Kafka**: Financial transactions (e.g., debit before credit).  
  - **Kinesis**: Clickstream sequences (e.g., page A → page B).  
- **Trade-off**:  
  - Ordering reduces parallelism (since keys must map to the same partition/shard).  

---

### **Interview Tip**  
*"Kafka and Kinesis guarantee ordering only within a partition/shard. For global ordering, you’d need a single partition (which limits throughput) or external sequencing (like a ledger)."*  



##Q8. **Priority Queue: Definition & Key Concepts**  
A **priority queue** is a specialized data structure where elements are processed based on **priority** (not just insertion order).  

#### **Key Characteristics (Bullet Points)**  
- **Not FIFO**: Unlike regular queues, the **highest-priority** element is removed first.  
- **Priority Assignment**: Each element has an associated priority (e.g., numerical value, urgency level).  
- **Operations**:  
  - **Insert**: Add an element with a priority.  
  - **Extract**: Remove the highest/lowest priority element.  
- **Implementation**: Typically uses a **heap** (binary heap) for efficient O(log n) inserts/extracts.  

#### **Real-World Examples**  
- **Hospital ER**: Critical patients (high priority) are treated before mild cases.  
- **CPU Scheduling**: High-priority tasks (e.g., system processes) run before background apps.  
- **Ride-Sharing**: Premium users get faster driver allocation.  


#### **Example in Python**  
```python
import heapq

pq = []
heapq.heappush(pq, (2, "Task A"))  # (priority, data)
heapq.heappush(pq, (1, "Task B"))  # Lower number = higher priority
print(heapq.heappop(pq)[1])         # Output: "Task B" (higher priority)
```

#### **Interview Tip**  
- Mention trade-offs: **Heaps** (O(log n)) vs. **sorted lists** (O(n) insert).  
- Use cases where order matters beyond FIFO (e.g., real-time systems).  



##Q9. **Process vs. Thread (Short Interview Answer)**  

- **Process**:  
  - Independent program instance with **separate memory** (isolated).  
  - Heavyweight (slower to create/switch).  
  - Crash in one process **doesn’t affect others**.  

- **Thread**:  
  - Lightweight **subset of a process**, shares memory/resources.  
  - Faster to create/switch (less overhead).  
  - Crash in one thread **can crash the entire process**.  

**Key Difference**:  
- Processes = **Isolated execution** (e.g., separate browser tabs).  
- Threads = **Parallel tasks within a process** (e.g., rendering + downloads in one tab).  


##Q10. **CAP Theorem: Short & Clear Explanation**  

**CAP Theorem** states that in a distributed system, you can only guarantee **two out of three** properties at the same time:  

1. **Consistency (C)**  
   - All nodes see the **same data at the same time** (e.g., a bank balance is accurate across all servers).  

2. **Availability (A)**  
   - Every request gets a **response (even if stale)**, with no system downtime.  

3. **Partition Tolerance (P)**  
   - The system keeps working **even if nodes disconnect** (e.g., network failure).  

#### **Trade-offs (Pick 2/3)**  
- **CA** (e.g., SQL databases): Sacrifice partition tolerance (single-server systems).  
- **CP** (e.g., MongoDB, Kafka): Sacrifice availability (e.g., reject requests if nodes can’t sync).  
- **AP** (e.g., Cassandra, DynamoDB): Sacrifice consistency (allow stale reads during partitions).  

### **Real-World Examples of CAP Theorem Trade-offs**  

#### **1. CP (Consistency + Partition Tolerance)**  
**Example: Banks & Financial Systems**  
- **Why?** Transactions **must** be consistent (e.g., $100 withdrawn can’t show $90 in one server and $100 in another).  
- **Sacrifice:** Availability (e.g., during a network partition, the system may reject transactions to avoid inconsistencies).  
- **Tech:** PostgreSQL, Kafka (for transactions), Zookeeper.  

#### **2. AP (Availability + Partition Tolerance)**  
**Example: Social Media (Facebook, Twitter)**  
- **Why?** Better to show **stale data** (e.g., delayed like counts) than fail to load the page.  
- **Sacrifice:** Consistency (temporary mismatches are acceptable).  
- **Tech:** Cassandra, DynamoDB, Redis (eventual consistency).  

#### **3. CA (Consistency + Availability)**  
**Example: Single-Node Databases (Rare in Distributed Systems)**  
- **Why?** No partitions (single server), so it’s always consistent and available.  
- **Sacrifice:** Partition tolerance (fails if the network splits).  
- **Tech:** SQLite, standalone MySQL (non-replicated).  

### **Key Insight**  
- **Most distributed systems choose **CP or AP** (since partitions are inevitable).  
- **CA systems** are rare in modern cloud architectures (they’re not fault-tolerant).  




##Q11. SQL Query to Find Percentage of Users Who Bought AirPods After iPhones

To solve this problem, we need to:
1. Identify users who bought an iPhone
2. Check if their next purchase was AirPods
3. Calculate the percentage of such users out of all iPhone buyers

Here's the SQL solution:

```sql
WITH UserPurchaseSequence AS (
    SELECT
        customer_id,
        product,
        transaction_timestamp,
        LEAD(product) OVER (PARTITION BY customer_id ORDER BY transaction_timestamp) AS next_product
    FROM transactions
),

iPhoneBuyers AS (
    SELECT DISTINCT customer_id
    FROM transactions
    WHERE product = 'iPhone'
),

iPhoneToAirPodsBuyers AS (
    SELECT DISTINCT customer_id
    FROM UserPurchaseSequence
    WHERE product = 'iPhone' AND next_product = 'AirPods'
)

SELECT
    (COUNT(DISTINCT iab.customer_id) * 100.0 /
    NULLIF(COUNT(DISTINCT ib.customer_id), 0) AS percentage
FROM iPhoneBuyers ib
LEFT JOIN iPhoneToAirPodsBuyers iab ON ib.customer_id = iab.customer_id;
```

## Explanation:

1. **UserPurchaseSequence CTE**:
   - Uses the `LEAD()` window function to see what each customer bought next
   - Partitions by customer and orders by timestamp to get chronological sequence

2. **iPhoneBuyers CTE**:
   - Identifies all unique customers who bought iPhones (denominator)

3. **iPhoneToAirPodsBuyers CTE**:
   - Finds customers whose immediate next purchase after iPhone was AirPods (numerator)

4. **Final Calculation**:
   - Divides the count of customers who bought AirPods after iPhones by total iPhone buyers
   - Multiplies by 100 to get percentage
   - Uses NULLIF to avoid division by zero


##Q12. Will the code work if you try to delete dictionary items during iteration?

### **Deleting Dictionary Items During Iteration in Python**

**Short Answer:**  
**No**, the code will **not** work correctly if you try to delete dictionary items during iteration. It raises a **`RuntimeError: dictionary changed size during iteration`**.

---

### **Why It Fails**
Python dictionaries track their size, and modifying them (adding/removing keys) while iterating breaks the internal iterator, causing an immediate error.

#### **Example of the Problem:**
```python
my_dict = {'a': 1, 'b': 2, 'c': 3}

for key in my_dict:
    if my_dict[key] % 2 == 0:
        del my_dict[key]  # 🚨 RuntimeError
```

---

### **How to Fix It**
#### **Option 1: Iterate Over a Copy of Keys**
```python
for key in list(my_dict.keys()):  # Explicit copy
    if my_dict[key] % 2 == 0:
        del my_dict[key]  # Safe
```

#### **Option 2: Store Keys to Delete First**
```python
keys_to_delete = [key for key, value in my_dict.items() if value % 2 == 0]
for key in keys_to_delete:
    del my_dict[key]
```

#### **Option 3: Use Dictionary Comprehension (Creates New Dict)**
```python
my_dict = {k: v for k, v in my_dict.items() if v % 2 != 0}
```

---

### **Key Takeaways**
- **Never modify a dict while iterating directly** over it.  
- **Safe methods**:  
  - Iterate over a **copy** of keys (`list(my_dict.keys())`).  
  - Record keys to delete first, then delete them.  
  - Use **dictionary comprehension** for filtering.  



In [ ]:
my_dict = {'a': 1, 'b': 2, 'c': 3}
for key in list(my_dict.keys()):  # Explicit copy
    if my_dict[key] % 2 == 0:
        del my_dict[key]
print( my_dict)

{'a': 1, 'c': 3}


##Q13. What is the difference between Pandas and PySpark?
### **Pandas vs. PySpark: Key Differences**

| **Feature**               | **Pandas**                                  | **PySpark**                                  |
|---------------------------|---------------------------------------------|---------------------------------------------|
| **Execution Environment** | Single-machine (RAM-limited)                | Distributed (cluster of machines)           |
| **Scalability**           | Handles data that fits in memory (~GBs)     | Handles TBs+ via parallel processing       |
| **Lazy Evaluation**       | No (immediate execution)                    | Yes (optimizes execution plan)              |
| **Fault Tolerance**       | No (crash = data loss)                      | Yes (via RDD lineage recovery)             |
| **Syntax**                | Pythonic (similar to NumPy)                 | SQL-like (with DataFrame API)               |
| **Use Cases**             | EDA, small datasets, single-node workflows  | Big data pipelines, ETL, cloud processing  |
| **Performance**           | Faster for small data (low overhead)        | Slower for tiny data (cluster setup cost)  |
| **Integration**           | Works with Python libraries (Matplotlib, etc.) | Integrates with Hadoop/Hive/S3/etc.       |

---

### **When to Use Which?**
- **Choose Pandas when:**
  - Data fits in memory (e.g., <10GB).
  - You need quick prototyping or EDA.
  - Your workflow uses Python ML libraries (scikit-learn, TensorFlow).

- **Choose PySpark when:**
  - Data exceeds memory (e.g., TBs of logs).
  - You need distributed processing (e.g., ETL pipelines).
  - Working in cloud environments (AWS EMR, Databricks).

---

### **Code Comparison**
#### **Pandas (Single-Node)**
```python
import pandas as pd
df = pd.read_csv("data.csv")
df_filtered = df[df["value"] > 100]  # In-memory operation
```

#### **PySpark (Distributed)**
```python
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read.csv("s3://bucket/data.csv")
df_filtered = df.filter(df["value"] > 100)  # Lazy evaluation
```

---

### **Key Technical Differences**
1. **Memory vs. Disk**:  
   - Pandas operates in memory.  
   - PySpark spills to disk when needed.  

2. **Parallelism**:  
   - Pandas: Single-threaded (unless using `modin.pandas`).  
   - PySpark: Automatically partitions data across nodes.  

---


##Q14. **Step-by-Step Approach to Migrate from Pandas to PySpark**  

#### **1. Assess the Current Pipeline**  
- **Identify bottlenecks**: Check RAM/CPU usage, slow transformations, and I/O operations.  
- **Document dependencies**: Note Pandas functions used (e.g., `.apply()`, `.groupby()`).  
- **Data volume**: Confirm if data size justifies PySpark (e.g., >10GB or growing).  

#### **2. Set Up PySpark Environment**  
- **Cluster configuration**: Choose # of nodes/cores based on data size (e.g., AWS EMR, Databricks).  
- **Dependencies**: Install `pyspark` and ensure compatibility with Python libraries (e.g., `numpy`, `scikit-learn`).  

#### **3. Rewrite Code Logic**  
- **Replace Pandas functions with PySpark equivalents**:  

  | **Pandas**               | **PySpark**                          |  
  |--------------------------|--------------------------------------|  
  | `df[df.col > 100]`       | `df.filter(df.col > 100)`            |  
  | `df.groupby().agg()`     | `df.groupBy().agg()` (*case-sensitive*) |  
  | `df.apply(func)`         | `df.withColumn("new", udf(func))`    |  
  | `pd.merge()`             | `df1.join(df2, on="key")`            |  

- **Handle lazy evaluation**: Call `.cache()` for reused DataFrames and `.collect()` only when needed.  

#### **4. Optimize Performance**  
- **Partitioning**: Repartition data to avoid skew (e.g., `df.repartition(100)`).  
- **Broadcast small DataFrames**: Use `broadcast()` for joins with small tables.  
- **Avoid UDFs**: Prefer built-in PySpark functions (e.g., `F.expr()` over Python UDFs).  

#### **5. Test and Validate**  
- **Unit tests**: Verify logic matches Pandas output (e.g., `assert df_pandas.equals(df_spark.toPandas())`).  
- **Scale testing**: Run on sample data → full dataset to catch memory/shuffle issues.  
- **Data integrity checks**: Compare row counts, NULL values, and key metrics.  

#### **6. Deploy and Monitor**  
- **Orchestration**: Schedule jobs via Airflow/Luigi (for batch) or Spark Streaming (for real-time).  
- **Monitoring**: Track Spark UI for slow tasks, skew, or spills to disk.  

---

### **Example Migration**  
**Pandas**:  
```python  
df = pd.read_csv("data.csv")  
df["discount"] = df["price"].apply(lambda x: x * 0.9)  
```  

**PySpark**:  
```python  
from pyspark.sql import functions as F  
df = spark.read.csv("data.csv")  
df = df.withColumn("discount", F.col("price") * 0.9)  # Avoid UDF for speed  
```  

---

# Mock 2:https://www.youtube.com/watch?v=E01qDvTjRic

# New Section

##Q1. Difference between RDD, Dataframe and Dataset.

> "All three — RDD, DataFrame, and Dataset — are abstractions provided by Apache Spark for working with distributed data. They differ mainly in **level of abstraction**, **performance optimization**, and **type safety**."

---

### ✅ RDD (Resilient Distributed Dataset)

> "RDD is the **most fundamental** and **low-level** abstraction in Spark.

It gives **fine-grained control** but **lacks built-in optimization**. There’s **no schema**, no catalyst optimizer, and you don’t get the benefits of Spark's engine-level tuning.

👉 **Use RDD when**:

* You need **fine control over data processing**
* Working with **unstructured data**
* Doing low-level transformations or legacy code

---

### ✅ DataFrame

> "DataFrame is a **higher-level abstraction** introduced in Spark 1.3. It’s like a distributed version of a SQL table, with rows and named columns. It’s backed by Spark SQL and **optimized by the Catalyst optimizer** which makes it **much faster** than RDDs.



👉 **Use DataFrame when**:

* You want **performance + ease of use**
* You're doing **ETL, reporting, or analytics**
* Schema is known and structured

---

### ✅ Dataset (Scala/Java Only)

> "Dataset sits in between RDD and DataFrame and combines the **benefits of both**. It’s **type-safe** like RDD and **optimized** like DataFrame.

Dataset is mainly used in **Scala and Java**, not Python (because Python lacks compile-time type safety)."

---

### 🧠 Interview Summary Line

> "Use **RDD** when you need low-level control and can trade off performance.
> Use **DataFrame** when you want better performance and simplicity with structured data.
> Use **Dataset** in Scala/Java when you need both **type safety** and **query optimization**."


##Q2. Different compression techniques such as snappy, biz2 and LZO.

> "In big data systems, compression helps reduce **storage costs** and **I/O time** — especially when dealing with petabytes of data. But not all compression algorithms are equal — some prioritize speed, while others focus on reducing size. Choosing the right one depends on the use case."

---

## 🔹 **Snappy**

> "Snappy is a compression algorithm developed by Google. It’s optimized for **speed** rather than maximum compression ratio. It’s widely used in Hadoop, Spark, and data lakes because it offers a good **tradeoff between compression and decompression speed**."

* **Fast compression and decompression**
* **Block-level compression** (great for columnar formats like Parquet)
* Ideal for **real-time and interactive workloads** (e.g., Spark jobs)
* Native support in **Parquet, ORC, Avro**

👉 Use Snappy when **performance and speed** are more important than storage savings.

---

## 🔹 **Bzip2**

> "Bzip2 focuses on **high compression ratio**, which means it compresses more aggressively but at the cost of **much slower speed**. It’s good when you're archiving data or trying to minimize space, but it's not suitable for interactive jobs."

* **Higher compression ratio** (smaller file size)
* **Slower** compression and decompression
* Not splittable by default — bad for Hadoop unless indexed
* Often used for **long-term storage or backups**

👉 Use Bzip2 when **space savings** matter more than speed (e.g., archival storage).

---

## 🔹 **LZO**

> "LZO is similar to Snappy in terms of **speed** — it’s also designed for **fast compression and decompression**, but with slightly better compression than Snappy in some cases. It’s splittable, which is useful in Hadoop ecosystems."

* **Fast like Snappy**
* Slightly better compression than Snappy, but less common
* **Splittable** — good for distributed processing (unlike Bzip2)
* Requires **extra setup** in Hadoop compared to Snappy

👉 Use LZO if you want speed **and** you need your files to be **splittable** for parallel processing in Hadoop.

---

## 🧠 Interview Summary Answer

> "Snappy and LZO are best for **fast processing and parallel computation**, making them ideal for Spark, Hive, or real-time pipelines. Bzip2 offers **better compression ratio** but is slow and not splittable, so it’s more suited for **cold storage or archival**.

In most production data pipelines, we use **Snappy with Parquet or ORC** formats because it's fast, lightweight, and fully supported across distributed systems."

---


##Q3. What is the write-ahead log?

> "A Write-Ahead Log (WAL) is a technique used in databases and distributed systems to ensure **data durability and crash recovery**.
> The core idea is: **Before applying any change to the actual data, the change is first recorded in a log.** This way, even if the system crashes during the update, it can replay the log and recover to a consistent state."

---

## 🔧 How It Works (Simple Steps)

1. A write request comes in.
2. The system **writes the change to the WAL (sequentially)**.
3. Only after the log is written, the **actual data is updated**.
4. If the system crashes, the WAL is used to **replay or roll back** operations.

---

## 📦 Where It’s Used

* **Relational databases** (e.g., PostgreSQL, MySQL InnoDB)
* **Distributed systems** (e.g., Apache Kafka, Apache Flink)
* **Big data tools** (e.g., Delta Lake, Apache HBase)
* **File systems** (e.g., HDFS Namenode)

---

## 🧠 Key Benefits

* ✅ **Durability**: Guarantees that committed transactions are not lost
* ✅ **Crash recovery**: Replay the WAL to restore consistent state
* ✅ **Performance**: Log writes are sequential, so they’re fast

---



## 🔥 Interview Soundbite

> "**Write-Ahead Logging** ensures durability in the face of failure. By recording every change in a log before applying it to the main data store, the system can always recover from crashes. It's a fundamental technique used in databases, file systems, and stream processing engines."

---



##Q4. Difference between SparkSession & SparkContext.

> "`SparkContext` is the **entry point to low-level Spark APIs**, mainly for working with RDDs.

`SparkSession` is the **unified entry point for all Spark functionalities** — including DataFrame, Dataset, and SQL operations. I

---


> "**Use `SparkSession` — it's the newer, more powerful API that wraps `SparkContext` and gives access to Spark SQL, DataFrames, and more.**"


##Q5. What are SQS and SNS in AWS?

> **"SQS (Simple Queue Service)** is a **message queue** used to decouple and buffer communication between distributed systems. It ensures reliable, asynchronous message delivery.

> **SNS (Simple Notification Service)** is a **pub/sub messaging system** that pushes messages to multiple subscribers like SQS queues, Lambda functions, or email endpoints in real time.

In short:
**SQS = pull-based queue** (decouples producers and consumers)
**SNS = push-based pub/sub** (broadcasts to multiple subscribers)"



##Q6. What is the work of Step functions in aws?


> "**Step Functions** is a fully managed **state machine service** on AWS for orchestrating serverless workflows,
> whereas **Apache Airflow** is an open-source **workflow scheduler** focused on **data pipelines** with more flexibility and extensibility."

---

### 🎯 **When to Use Each**

* **AWS Step Functions**:

  * Best for **event-driven**, **serverless**, and tightly integrated **AWS workflows**
  * Great for orchestrating **Lambda, ECS, Batch, SageMaker**, etc.
  * No infrastructure to manage; scales automatically
  * Visual interface with built-in **retry, timeout, and error handling**
  * Declarative (JSON/YAML-based) — limited logic control

* **Apache Airflow**:

  * Best for **complex data workflows**, **ETL pipelines**, or anything requiring **custom Python logic**
  * Highly **extensible** via Python operators/plugins
  * Can run **any task** (not just AWS)
  * Requires infrastructure setup and maintenance (can be managed via MWAA or Astronomer)
  * Fine-grained control over **scheduling**, **dependencies**, and **dynamic DAGs**

---

> "**Step Functions** are ideal for orchestrating AWS-native, serverless workflows with minimal setup.
> **Airflow** offers more flexibility and power for building complex, cross-platform **data pipelines**, but needs more management.
> I’d choose Step Functions for quick AWS integrations and Airflow when I need rich DAG logic or hybrid workflows."



##Q7. How do you decide which should go to the Data Warehouse and which should be treated as an external table?


> "If the data is **critical for analytics, reporting, or BI**, and needs to be **clean, structured, and performant**, it should go into the **Data Warehouse**.
> If the data is **raw, semi-structured, very large, or infrequently queried**, it’s better to keep it as an **external table**—usually on data lake storage like S3 or HDFS—so it's cheaper to store and more flexible to process."

Excellent follow-up. Here's how you can explain what an **external table** means in the context of the previous question — especially for an interview.

---

## 🔍 What Is an External Table?

> "An **external table** is a table definition that points to **data stored outside the data warehouse**, typically in a data lake or file system like **Amazon S3, HDFS, or Azure Data Lake**.
> The table only stores the **schema and metadata**, but not the data itself."

In other words:

* You're **not importing** the data into the warehouse
* You're **querying it where it already lives** (e.g., Parquet/CSV files in S3)

---

## 📘 Example (e.g., Hive, Athena, BigQuery External Table)

```sql
CREATE EXTERNAL TABLE clickstream_logs (
  user_id STRING,
  timestamp TIMESTAMP,
  page_url STRING
)
STORED AS PARQUET
LOCATION 's3://my-bucket/logs/';
```

* This **defines a schema**, but doesn’t move the data.
* Querying this table reads directly from the files in S3.

---

## 💡 Key Traits of External Tables

* **Data remains in external storage**
* Defined using **metadata only**
* Often **used for raw or semi-structured data**
* **Schema-on-read** (applied at query time, not load time)
* Can be dropped **without deleting the underlying data**

---

## 🧠 Interview Summary Answer

> "An **external table** allows you to query data where it lives — in your data lake — without moving it into the warehouse. It's useful when dealing with large volumes of raw, semi-structured, or infrequently used data. You define the schema in your SQL engine, but the data stays outside."


##Q8. How do you choose the database in your project - relational and non-relational?

**"I choose a relational database when my data is structured, requires ACID transactions, strong consistency, and complex joins—like in financial or transactional systems.**

**I prefer a non-relational (NoSQL) database when I need to handle unstructured or semi-structured data, require high scalability, flexible schemas, or need fast reads/writes—such as in caching, real-time analytics, or storing documents or key-value data."**


##Q9. What is elastic search
Elasticsearch is a powerful search engine that helps you quickly find and analyze large amounts of data, like text documents or logs. It’s built to be fast, scalable, and easy to use for searching through data in real-time.

##Q10. Difference between windows and group by.

> "`GROUP BY` is used when you want to aggregate rows – for example, calculating the total sales per customer. But it collapses rows – you lose individual detail.

Window functions, on the other hand, allow you to perform calculations **across a set of rows** that are related to the current row, **without collapsing** the data. They’re great for running totals, ranking, percentiles, and time-based comparisons.

So, if I want to know each employee’s salary and their **rank within their department**, I’d use a window function. If I only want the **total salary per department**, I’d use `GROUP BY`."

---

## ✅ When to Use Each

| Use Case                               | Use                                       |
| -------------------------------------- | ----------------------------------------- |
| Total sales per product                | `GROUP BY product_id`                     |
| Each sale with total sales per product | `SUM(...) OVER (PARTITION BY product_id)` |
| Customer with max purchase             | `GROUP BY + MAX()`                        |
| Every purchase + rank of customer      | `RANK() OVER (PARTITION BY ...)`          |
| Monthly sales trend with lag           | `LAG()/LEAD()` with `OVER(...)`           |

---

## 🧠 Summary Answer for Interview

> "Use `GROUP BY` when you want to summarize or aggregate data into fewer rows.
> Use **window functions** when you need aggregate-like operations but still want to **retain the individual row details**.
> Window functions are more powerful for **analytics and time-series use cases**, while `GROUP BY` is best for **summary reports**."



##Q11. Difference between having and where.

> “`WHERE` is more efficient because it reduces the number of rows before aggregation. `HAVING` should only be used when filtering aggregated results.”

---

> "`WHERE` filters raw rows before any aggregation or grouping happens. `HAVING` is used to filter results **after** aggregation, such as filtering groups based on `SUM` or `COUNT`. For performance reasons, we should always use `WHERE` when possible to reduce the dataset early."



## 🔍 **1. Conceptual Difference**

| Aspect              | `WHERE` Clause                      | `HAVING` Clause                           |
| ------------------- | ----------------------------------- | ----------------------------------------- |
| Applies to          | **Rows** before grouping            | **Groups** after aggregation (`GROUP BY`) |
| Filters             | Individual rows                     | Aggregated results                        |
| Used with           | `SELECT`, `UPDATE`, `DELETE`        | Usually with `GROUP BY`                   |
| Aggregate functions | ❌ Cannot use (`SUM`, `COUNT`, etc.) | ✅ Can use                                 |

---

## ⚙️ **2. Execution Order (Simplified)**

SQL Query Execution Order (conceptual):

```sql
FROM → WHERE → GROUP BY → HAVING → SELECT → ORDER BY
```

So:

* `WHERE` filters **before grouping**.
* `HAVING` filters **after aggregation**.

---

## 📊 **3. Example**

Let’s say we have a `sales` table:

| id | customer | region | amount |
| -- | -------- | ------ | ------ |
| 1  | Alice    | East   | 100    |
| 2  | Bob      | West   | 200    |
| 3  | Alice    | East   | 300    |
| 4  | Bob      | West   | 150    |
| 5  | Eve      | East   | 120    |

---

### 🧱 Example with `WHERE`:

**Question**: Get total sales per customer, but only for rows where `region = 'East'`.

```sql
SELECT customer, SUM(amount) AS total_sales
FROM sales
WHERE region = 'East'
GROUP BY customer;
```

**Explanation**:

* `WHERE` filters out non-East rows first.
* Then `GROUP BY` and `SUM` are applied.

---

### 🧱 Example with `HAVING`:

**Question**: Get total sales per customer, but **only show** customers with **total sales > 200**.

```sql
SELECT customer, SUM(amount) AS total_sales
FROM sales
GROUP BY customer
HAVING SUM(amount) > 200;
```

**Explanation**:

* All rows are grouped first.
* `HAVING` filters **after** the aggregation.

---

### 🧱 Combined Example:

```sql
SELECT customer, SUM(amount) AS total_sales
FROM sales
WHERE region = 'East'
GROUP BY customer
HAVING SUM(amount) > 200;
```

This filters only "East" rows, groups by customer, and returns only those with > 200 sales in East.

---

## ⚡ **4. Performance Difference**

| Factor          | `WHERE`                           | `HAVING`                             |
| --------------- | --------------------------------- | ------------------------------------ |
| Execution Time  | **Faster** (early filtering)      | **Slower** (post-aggregation)        |
| Optimization    | Better optimized by SQL engines   | Less optimized unless indexed fields |
| Recommended Use | Always filter early using `WHERE` | Use `HAVING` **only when needed**    |


##Q12. Difference between groupby and reduceby.

### 🧠 **How to Explain to an Interviewer**

> "Both `groupByKey` and `reduceByKey` are used to perform aggregations by key in Spark, but they differ significantly in performance and use case.

* `groupByKey` groups all the values with the same key into a single list—*without doing any aggregation*. This causes a **full shuffle** of data, moving all values across the network, which is expensive and can lead to **out-of-memory** errors.

* `reduceByKey`, on the other hand, performs a **local reduction (or combine)** on each partition *before* shuffling. This means **less data is moved**, and aggregation is more efficient.

That’s why **reduceByKey is preferred** when performing aggregation, like sum, count, or average.
Only use `groupByKey` when you need **all the raw values** for a key, like for custom sorting or full downstream processing."

---

### 🧪 Example

```python
# Dataset: [(k, v), (k, v), ...]
data = sc.parallelize([
    ("a", 1),
    ("b", 2),
    ("a", 3),
    ("b", 4)
])

# groupByKey (collects all values)
grouped = data.groupByKey()
# Output: ("a", [1, 3]), ("b", [2, 4])

# reduceByKey (aggregates during shuffle)
reduced = data.reduceByKey(lambda x, y: x + y)
# Output: ("a", 4), ("b", 6)
```

---

### 💡 Rule of Thumb

> **"Use `reduceByKey` when you can aggregate. Use `groupByKey` only if you absolutely need all values per key."**


##Q13. System Design and Data Model: Messaging system

We are given a conversation where a system design and data modeling problem is discussed. The problem is about designing a system to monitor employee communications across multiple platforms (like Slack, Zoom, Microsoft Teams, email) for potential fraud or harassment. The goal is to create a product (data pipeline) that extracts data from these sources, processes it, and stores it in a way that allows for easy access by legal teams or others when needed. The system should handle both batch and streaming data.
Key requirements:
1. **Multiple Sources**: Slack, Zoom, Microsoft Teams, email (Outlook), etc.
2. **Data Volume**: Large volume of data (chat logs, emails).
3. **Processing**: Basic transformations to filter for unethical content (harassment, fraud) and prepare data for generative AI (like creating embeddings for similarity search).
4. **Storage**: Data should be stored in different zones (raw, transformed) for consumption by data scientists or legal teams.
5. **Access**: The system must allow exporting data securely to external parties (like lawyers).
The conversation also covers data modeling and the overall system design, including:
- Using cloud services (AWS, GCP, etc.)
- Data extraction methods (APIs, CDC, Kafka, Kinesis)
- Data storage (S3 for raw, Parquet for processed data)
- Data processing (Spark, AWS Glue)
- Data modeling (star schema with fact and dimension tables)
- Data exposure (APIs, secure sharing, CSV exports)
Now, let's break down the questions and provide the best answers.
### I. System Design Questions and Best Answers
1. **How would you design the data ingestion for multiple sources (Slack, Teams, Zoom, email) that may use different protocols (APIs, CDC, etc.)?**
   - **Answer**:
     - Use a combination of batch and streaming ingestion.
     - For streaming sources (e.g., Slack messages in real-time), use Kafka or AWS Kinesis to capture data and land it in a raw data store (like S3).
     - For batch sources (e.g., daily email dumps), use scheduled jobs (Airflow, AWS Glue) to extract via APIs and store in S3.
     - For email (Outlook), use Microsoft Graph API to pull emails in batch or set up a streaming connector if possible.
     - Ensure idempotency and handle duplicates.
2. **How would you structure the storage (data lake) for this system?**
   - **Answer**:
     - Use a medallion architecture:
       - **Bronze/Raw Zone**: Store raw data in S3 in the original format (JSON, CSV, etc.). Partition by date/source for efficient querying.
       - **Silver/Cleansed Zone**: Convert data to Parquet format. Apply basic cleansing (e.g., deduplication, schema enforcement). Partition by date/source.
       - **Gold/Curated Zone**: Store modeled data (fact and dimension tables in Parquet/ORC) for consumption. This is where we apply business logic (e.g., flagging unethical messages).
3. **How would you process the data to detect unethical content?**
   - **Answer**:
     - Use Spark (AWS Glue, EMR) for batch processing and Spark Streaming for real-time.
     - Implement transformation logic:
       - Tokenize text and match against a list of unethical keywords (e.g., from a company's code of conduct).
       - Use NLP techniques (e.g., sentiment analysis, entity recognition) for advanced detection.
       - For generative AI preparation, create embeddings (using models like BERT) and store them in a vector database (e.g., FAISS, Pinecone) for similarity search.
     - Output: A fact table with message details and a flag for unethical content.
4. **How would you model the data for analysis?**
   - **Answer**:
     - Use a star schema:
       - **Fact Table**: `message_fact`
         - message_id (PK), sender_id (FK), receiver_id (FK), text_id (FK), date_id (FK), time_id (FK), channel_id (FK), is_unethical (boolean)
       - **Dimension Tables**:
         - `employee_dim`: employee_id (PK), name, email, department, etc.
         - `text_dim`: text_id (PK), message_text, language, etc. (for text analysis)
         - `date_dim`: date_id (PK), date, day, month, year, etc.
         - `time_dim`: time_id (PK), hour, minute, etc.
         - `channel_dim`: channel_id (PK), platform_name (e.g., Slack), channel_name
     - This allows efficient filtering (e.g., "show all unethical messages by employee X in June").
5. **How would you handle data quality and orchestration?**
   - **Answer**:
     - **Orchestration**: Use Airflow or AWS Step Functions to manage workflows (ingestion, processing, validation).
     - **Data Quality**:
       - At ingestion: Validate schema, non-null keys.
       - After transformations: Check for duplicates, nulls, and business rules (e.g., unethical flag rate).
       - Tools: Great Expectations, AWS Deequ, or custom Spark checks.
6. **How would you expose the data to external parties (e.g., lawyers) securely?**
   - **Answer**:
     - **Internal Consumption**: Use a data warehouse (Redshift, Snowflake) for internal users. Create views for filtered data (e.g., by employee, date).
     - **External Sharing**:
       - Option 1: Generate encrypted CSV/Parquet files and share via a secure link (AWS S3 pre-signed URLs) with expiration. Notify via email.
       - Option 2: Use Snowflake's data sharing feature (if using Snowflake) to share a read-only view with the external party.
       - Option 3: Build a secure API (using API Gateway and Lambda) that returns filtered data. Authenticate via API keys (rotated regularly) and use TLS encryption.
       - Security: Ensure data is encrypted in transit (TLS) and at rest (AES-256). Mask PII if needed.
### II. Data Modeling Questions and Best Answers
1. **What are the key entities and relationships?**
   - **Answer**:
     - Entities: Employee, Message, Text, Date, Time, Channel.
     - Relationships:
       - A Message is sent by one Employee (sender) and received by one Employee (receiver).
       - A Message has one Text content.
       - A Message is sent on one Date and Time.
       - A Message is sent via one Channel.
2. **How would you design the `text_dim` table to support text analysis?**
   - **Answer**:
     - `text_dim` table:
       - text_id (PK)
       - raw_text (original text)
       - cleaned_text (after removing special characters, etc.)
       - language
       - embedding_vector (for generative AI, stored as an array of floats)
     - This allows efficient text search and similarity matching.
3. **Should we store the unethical flag in the fact or dimension table?**
   - **Answer**: Store it in the fact table (`message_fact.is_unethical`). This is a property of the message event, not the text itself (the same text could be ethical in one context and unethical in another).
4. **How would you handle slowly changing dimensions (SCD) for employee data?**
   - **Answer**:
     - Use SCD Type 2 for `employee_dim` to track changes (e.g., department changes). Add columns:
       - start_date, end_date, is_current
     - Example: If an employee moves from Sales to Marketing, insert a new row with the new department and update the old row's end_date.
5. **How would you optimize the fact table for large-scale querying?**
   - **Answer**:
     - Partition by date_id and channel_id.
     - Use columnar storage (Parquet/ORC) for compression and efficient scans.
     - Cluster by sender_id and is_unethical for common filters.
### Summary of Best Practices
- **Extraction**: Use Kafka/Kinesis for streaming, batch APIs for others.
- **Storage**: S3 with medallion architecture (bronze, silver, gold).
- **Processing**: Spark for batch/streaming, with NLP for unethical content detection.
- **Modeling**: Star schema with fact and dimension tables.
- **Exposure**: Secure APIs, pre-signed URLs, or data sharing features.
- **Security**: Encryption (TLS, AES-256), API keys, and access controls.
This design meets scalability, security, and functional requirements.

##Q14. What libraries have you used in Python in Data engineering.

1. **Data Ingestion**

   * Read data from various sources:

     ```python
     pd.read_csv(), pd.read_json(), pd.read_sql(), pd.read_parquet(), pd.read_excel()
     ```
   * Example:

     ```python
     df = pd.read_csv("sales.csv")
     ```

2. **Data Cleaning & Transformation**

   * Handle missing values: `df.dropna()`, `df.fillna()`
   * Rename columns: `df.rename()`
   * Type conversion: `df.astype()`
   * Replace values: `df.replace()`
   * Filtering & slicing: `df[df['col'] > 10]`
   * Example:

     ```python
     df['date'] = pd.to_datetime(df['date'])
     df = df.dropna(subset=['price'])
     ```

3. **ETL (Extract, Transform, Load)**

   * Perform joins, merges: `pd.merge()`, `df.join()`
   * Grouping & aggregation: `df.groupby().agg()`
   * Sorting, deduplication: `df.sort_values()`, `df.drop_duplicates()`

4. **Exploratory Data Analysis (EDA)**

   * Summary stats: `df.describe()`, `df.info()`
   * Correlation matrix: `df.corr()`
   * Value counts: `df['category'].value_counts()`

5. **Exporting Data**

   * Save to file or database:

     ```python
     df.to_csv(), df.to_parquet(), df.to_sql()
     ```

6. **Interfacing with Databases**

   * Read/write using `SQLAlchemy` or `sqlite3`:

     ```python
     from sqlalchemy import create_engine
     engine = create_engine('sqlite:///mydb.sqlite')
     df.to_sql('table_name', engine)
     ```

---

### ⚠️ Limitations of Pandas in Data Engineering

* **Memory-bound**: Not suitable for large datasets that don’t fit into memory.
* **Not distributed**: Single-machine only; use **Dask**, **PySpark**, or **Polars** for larger datasets.

Here’s a compact guide to **NumPy for Data Engineering**, with practical code examples that are commonly useful in data pipelines, ETL, and preprocessing tasks.

---

### Numpy
```python
import numpy as np

# Create arrays
arr = np.array([1, 2, 3, 4])
matrix = np.array([[1, 2], [3, 4]])

# Create arrays filled with zeros, ones, or random numbers
zeros = np.zeros((3, 3))
ones = np.ones((2, 2))
rand = np.random.rand(2, 3)
```

---

### 🧮 2. **Basic Array Operations**

```python
arr = np.array([10, 20, 30, 40])

# Element-wise operations
arr2 = arr * 2         # [20, 40, 60, 80]
arr3 = arr + 5         # [15, 25, 35, 45]

# Aggregate operations
mean_val = np.mean(arr)
sum_val = np.sum(arr)
```

---

### 🔄 4. **Reshaping and Merging**

```python
a = np.array([1, 2, 3, 4, 5, 6])

# Reshape
reshaped = a.reshape((2, 3))

# Concatenate
b = np.array([7, 8, 9])
concat = np.concatenate((a, b))
```

---



##Q15. Different functions in Spark

## 🔹 1. **Reading and Writing Data**

```python
# Reading
spark.read.csv("path", header=True, inferSchema=True)
spark.read.json("path")
spark.read.parquet("path")
spark.read.orc("path")
spark.read.format("jdbc").options(...).load()

# Writing
df.write.csv("path", mode="overwrite")
df.write.parquet("path")
df.write.json("path")
df.write.mode("append").saveAsTable("table_name")
```

---

## 🔹 2. **DataFrame Transformations**

```python
df.select("col1", "col2")                   # select columns
df.filter(df["col"] > 100)                  # filter rows
df.withColumn("new_col", df["col"] + 1)     # add/modify column
df.drop("col_to_drop")                      # drop column
df.distinct()                               # remove duplicates
df.dropDuplicates(["col1", "col2"])         # remove based on columns
df.limit(10)                                # limit rows
```

---

## 🔹 3. **Aggregations and Grouping**

```python
from pyspark.sql import functions as F

df.groupBy("col").agg(F.sum("amount"), F.avg("score"))
df.agg(F.max("col"), F.min("col"))
df.count()
```

---

## 🔹 4. **Joins**

```python
df1.join(df2, df1["id"] == df2["id"], "inner")
df1.join(df2, ["id"], "left_outer")  # also: right_outer, full_outer, left_semi, left_anti
```

---

## 🔹 5. **Window Functions**

```python
from pyspark.sql.window import Window

windowSpec = Window.partitionBy("col1").orderBy("col2")
df.withColumn("rank", F.rank().over(windowSpec))
df.withColumn("row_num", F.row_number().over(windowSpec))
df.withColumn("lag_value", F.lag("value", 1).over(windowSpec))
```

---

## 🔹 6. **Handling Nulls**

```python
df.dropna()                            # drop rows with nulls
df.fillna({"col1": 0, "col2": "N/A"})  # fill nulls
df.na.replace("old", "new")            # replace nulls or values
```

---

## 🔹 7. **UDFs (User Defined Functions)**

```python
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

def custom_func(x):
    return x * 2

double_udf = udf(custom_func, IntegerType())
df.withColumn("doubled", double_udf(df["col"]))
```

> ⚠️ Use built-in functions instead of UDFs when possible—they're faster and optimized.

---

## 🔹 8. **Spark SQL**

```python
df.createOrReplaceTempView("my_table")
spark.sql("SELECT col1, COUNT(*) FROM my_table GROUP BY col1")
```

---

## 🔹 9. **Repartitioning and Caching**

```python
df.repartition(10)      # shuffle-based
df.coalesce(2)          # reduce partitions (no shuffle)
df.cache()              # in-memory cache
df.persist()            # cache with storage level
```

---

## 🔹 10. **Writing to Hive / External Tables**

```python
df.write.saveAsTable("db.table_name")      # Hive table
df.write.option("path", "hdfs_path").saveAsTable("external_table")
```

---

## 🔹 11. **Schema and Metadata**

```python
df.printSchema()
df.schema
df.dtypes
df.columns
```

---

## 🔹 12. **Data Types and Casting**

```python
df.withColumn("col", df["col"].cast("int"))
F.col("col").cast("string")
```

---


## Bonus: **Performance & Debugging**

```python
df.explain(True)           # physical plan
df.rdd.getNumPartitions()  # partition count
df.show(5, truncate=False) # display rows
```

Great question! Beyond the most **commonly used Spark functions** listed earlier, Spark provides a **rich set of additional functions** (especially under `pyspark.sql.functions`) that are helpful in **data cleansing, ETL, feature engineering, and analytics**.

Here’s a categorized list of **additional but important Spark functions** that are **less commonly covered but useful in real-world data engineering**:

---

## 🔸 String Functions

```python
F.concat(F.col("col1"), F.col("col2"))       # Concatenate columns
F.substring("col", 1, 3)                     # Substring
F.upper("col"), F.lower("col")              # Change case
F.trim("col"), F.ltrim("col"), F.rtrim("col")
F.length("col")                              # String length
F.instr("col", "substr")                     # Index of substring
F.replace("col", "old", "new")               # Replace substring
F.regexp_replace("col", pattern, "new")      # Regex replace
F.split("col", ",")                          # Split into array
F.concat_ws("-", "col1", "col2")             # Concatenate with separator
```

---

## 🔸 Date and Time Functions

```python
F.current_date(), F.current_timestamp()
F.date_format("timestamp_col", "yyyy-MM-dd")
F.year("col"), F.month("col"), F.dayofmonth("col")
F.hour("col"), F.minute("col"), F.second("col")
F.date_add("date", 7), F.date_sub("date", 7)
F.datediff("end_date", "start_date")
F.to_date("timestamp_col")
F.to_timestamp("string_col", "yyyy-MM-dd HH:mm:ss")
```

---

## 🔸 Array & Map Functions

```python
F.array("col1", "col2")                      # Create array
F.size("array_col")                          # Length of array
F.array_contains("array_col", "value")
F.explode("array_col")                       # Explode array into rows
F.map_keys("map_col"), F.map_values("map_col")
F.element_at("array_col", 1)                 # Get element by index
```

---

## 🔸 Conditional Logic

```python
F.when(F.col("age") > 18, "adult").otherwise("minor")
F.expr("CASE WHEN col > 10 THEN 'high' ELSE 'low' END")
F.isnull("col"), F.isnan("col")              # Check for null or NaN
```

---

## 🔸 JSON Functions

```python
F.from_json("json_col", schema)              # Parse JSON
F.to_json("struct_col")                      # Convert struct to JSON
F.get_json_object("json_col", "$.field")     # Extract value from JSON
```

---

## 🔸 Hashing & Encryption

```python
F.sha2("col", 256)
F.md5("col")
F.crc32("col")
```

---

## 🔸 Data Skew & Sampling

```python
df.sample(withReplacement=False, fraction=0.1)
df.randomSplit([0.7, 0.3])
```

---

## 🔸 Miscellaneous Useful Functions

```python
F.monotonically_increasing_id()              # Unique ID generator
F.input_file_name()                          # Add source file name as column
F.broadcast(df)                              # Broadcast join optimization
F.struct("col1", "col2")                     # Combine columns into struct
```

---

## 🔸 Partition Functions

```python
df.write.partitionBy("col").parquet("path")
```
---

### ✅ Tip:

You can explore all functions with:

```python
import pyspark.sql.functions as F
dir(F)  # Lists all available functions
```



##Q16. **Null** and **NaN** with examples:

---

## What is **NaN**?

**NaN** stands for **"Not a Number"**. It is a special floating-point value defined by the IEEE 754 standard to represent:

* Undefined or unrepresentable numeric results
* Missing numeric values distinct from `null`
* Results of invalid operations like `0/0`, `∞ - ∞`, or `sqrt(-1)` in some contexts

---

## Characteristics of NaN:

* NaN is a **float/double** type value.
* NaN is **not equal** to anything, **including itself**. So `NaN == NaN` is `False`.
* Used to indicate invalid or missing numeric data without being a null.
* Common in scientific computing and data processing.

---

## Examples of NaN in Python/Spark context:

| Expression                    | Result                   | Explanation                       |
| ----------------------------- | ------------------------ | --------------------------------- |
| `float('nan')`                | `NaN`                    | Explicit NaN value                |
| `0.0 / 0.0`                   | `NaN`                    | Division zero by zero             |
| `math.sqrt(-1)`               | Error or NaN             | Imaginary number (depends on lib) |
| `float('inf') - float('inf')` | `NaN`                    | Infinity minus infinity           |
| Missing float value           | Often represented as NaN | Numeric missing value             |

---

## Example in PySpark:

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, isnull, col

spark = SparkSession.builder.getOrCreate()

data = [(1.0,), (float('nan'),), (None,), (3.5,)]
df = spark.createDataFrame(data, ["value"])

df.show()
# +-----+
# |value|
# +-----+
# |  1.0|
# |  NaN|
# | null|
# |  3.5|
# +-----+

# Filter rows where value is NaN
df.filter(isnan(col("value"))).show()
# +-----+
# |value|
# +-----+
# |  NaN|
# +-----+
```

---

## Summary:

| Null                                | NaN                                       |
| ----------------------------------- | ----------------------------------------- |
| Represents missing data or no value | Represents an invalid or undefined number |
| Applies to all data types           | Only for floating-point types             |
| Spark: `F.isnull()`                 | Spark: `F.isnan()`                        |
| `None` in Python                    | `float('nan')` in Python                  |

---


##Q17. Handling JSON

## 🔹 1. **Standard JSON File (JSON Lines / Line-delimited)**

Each line is a valid JSON object.

### 📁 Example (file: `data1.json`)

```json
{"name": "Alice", "age": 25}
{"name": "Bob", "age": 30}
```

### ✅ Spark Code

```python
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Read JSON").getOrCreate()

df = spark.read.json("data1.json")
df.show()
```

---

## 🔹 2. **Multi-line JSON File (Single JSON array or nested object)**

Used when a full JSON object spans multiple lines.

### 📁 Example (file: `data2.json`)

```json
[
  {"name": "Alice", "age": 25},
  {"name": "Bob", "age": 30}
]
```

### ✅ Spark Code

```python
df = spark.read.option("multiLine", True).json("data2.json")
df.show()
```

---

## 🔹 3. **Nested JSON**

Used when fields are nested JSON objects or arrays.

### 📁 Example

```json
{
  "name": "Alice",
  "address": {
    "city": "NYC",
    "zip": "10001"
  }
}
```

### ✅ Spark Code

```python
df = spark.read.option("multiLine", True).json("nested.json")
df.printSchema()

# Access nested fields
df.select("name", "address.city", "address.zip").show()
```

---

## 🔹 4. **Array of JSON Objects**

Sometimes JSON file contains just an array.

### 📁 Example

```json
[
  {"id": 1, "value": "a"},
  {"id": 2, "value": "b"}
]
```

### ✅ Spark Code

```python
df = spark.read.option("multiLine", True).json("array.json")
df.show()
```

If Spark complains about a mismatch (e.g. expecting an object not an array), you can wrap it into a Dataset:

```python
from pyspark.sql.functions import explode

df = spark.read.option("multiLine", True).json("array.json")
df = df.select(explode(df).alias("item")).select("item.*")
df.show()
```

---

## 🔹 5. **Corrupt / Mixed JSON (bad lines)**

Use `mode` and `columnNameOfCorruptRecord` options.

### ✅ Spark Code

```python
df = spark.read.option("mode", "PERMISSIVE") \
               .option("columnNameOfCorruptRecord", "_corrupt_record") \
               .json("corrupt.json")

df.show(truncate=False)
```

Other `mode` options:

* `"PERMISSIVE"` (default): corrupt records are in a new column.
* `"DROPMALFORMED"`: drops bad rows.
* `"FAILFAST"`: fails on first bad row.

---

## 🔹 6. **Reading from JSON String (not file)**

```python
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True)
])

json_str = ['{"name":"Alice", "age":25}', '{"name":"Bob", "age":30}']
rdd = spark.sparkContext.parallelize(json_str)
df = spark.read.schema(schema).json(rdd)
df.show()
```

---

## 📌 Bonus: Write DataFrame to JSON

```python
df.write.mode("overwrite").json("output/path")
```

Options:

* `mode("overwrite")`, `mode("append")`, etc.
* `.option("compression", "gzip")` if needed

---


##Q18. JSON File Flattening

Flattening a **nested JSON file** in Spark is a common task—especially for data engineers dealing with hierarchical or semi-structured data (like logs, API responses, etc.).

---

## ✅ Goal of Flattening

Convert this:

```json
{
  "name": "Alice",
  "address": {
    "city": "NYC",
    "zip": "10001"
  },
  "contacts": [
    {"type": "email", "value": "alice@example.com"},
    {"type": "phone", "value": "1234567890"}
  ]
}
```

To this:

| name  | address.city | address.zip | contacts.type | contacts.value |
| ----- | ------------ | ----------- | ------------- | -------------- |
| Alice | NYC          | 10001       | email         | alice\@...     |
| Alice | NYC          | 10001       | phone         | 1234567890     |

---

## 🔧 Step-by-Step PySpark Code to Flatten JSON

### 1. **Read and Inspect JSON**

```python
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Flatten JSON").getOrCreate()

df = spark.read.option("multiline", True).json("nested.json")
df.printSchema()
df.show(truncate=False)
```

---

### 2. **Flatten Structs (Nested Objects)**

```python
from pyspark.sql.functions import col

flattened_df = df.select(
    col("name"),
    col("address.city").alias("address_city"),
    col("address.zip").alias("address_zip")
)
flattened_df.show()
```

---

### 3. **Flatten Arrays (Explode Contacts)**

```python
from pyspark.sql.functions import explode

contacts_df = df.select(
    "name",
    col("address.city").alias("address_city"),
    col("address.zip").alias("address_zip"),
    explode("contacts").alias("contact")
)

flattened_df = contacts_df.select(
    "name", "address_city", "address_zip",
    col("contact.type").alias("contact_type"),
    col("contact.value").alias("contact_value")
)

flattened_df.show()
```

---


## 💾 Writing Flattened Data

```python
flat_df.write.mode("overwrite").json("flattened_output/")
```



##Q19. Client Mode and Cluster Mode.


> “In **client mode**, the driver runs on the same machine that submits the Spark job, which is useful during development and testing when you need quick feedback or access to logs. In **cluster mode**, the driver is launched on a node inside the cluster, making it more resilient and better suited for production jobs that need to run reliably without relying on the submitter's machine.”


### ✅ **1. Client Mode**

* **Driver Location**: The **driver runs on the machine** that submits the Spark job (typically your laptop or an edge node).
* **Executors**: Run on the cluster (e.g., YARN, Kubernetes, or standalone).
* **Communication**: The client machine must stay **active and reachable** because the cluster communicates back with the driver.

#### 🟩 Use Case:

* Good for **development**, **debugging**, and **interactive analysis**.
* Preferred when you're **submitting jobs from a local or edge machine** with stable network access.

#### ⚠️ Limitation:

* If the client goes down, the whole job fails.
* Not suitable for **long-running jobs** or **unreliable client machines**.

---

### ✅ **2. Cluster Mode**

* **Driver Location**: The **driver runs on one of the nodes** in the cluster (not on the client machine).
* **Executors**: Run on other nodes in the cluster as usual.
* **Client’s Role**: Submits the job and **exits**; it’s not involved in job execution afterward.

#### 🟩 Use Case:

* Ideal for **production workloads** and **scheduled jobs**.
* Used when you want **reliability** and don’t want the job to depend on your local machine.

#### ⚠️ Limitation:

* Harder to debug compared to client mode, especially in real-time.

---

### 🧠 Key Difference Summary:

| Feature            | Client Mode                       | Cluster Mode                         |
| ------------------ | --------------------------------- | ------------------------------------ |
| Driver runs on     | Client (submitter) machine        | Worker node in the cluster           |
| Network dependency | Needs stable connection to client | No dependency on client after submit |
| Debugging          | Easier (logs on client)           | Harder (logs on cluster)             |
| Use case           | Interactive / development         | Production / scheduled workloads     |

---

##Q20.